In [1]:
from bigdatavqa.datautils import DataUtils
from bigdatavqa.coreset import Coreset, normalize_np
from qiskit.opflow.primitive_ops import PauliSumOp
from numpy.linalg import inv
import numpy as np
from cudaq import spin

/tmp/ipykernel_6980/2119346348.py:3: DeprecationWarning: The ``qiskit.opflow`` module is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/opflow_migration.
  from qiskit.opflow.primitive_ops import PauliSumOp


In [2]:
number_of_qubits = 5
circuit_depth = 1
max_shots = 100
max_iterations = 100

max_iterations = 100
number_of_runs = 100
size_vec_list = 10


In [3]:
def get_raw_data(data_location):
    data_utils = DataUtils(data_location)

    try:
        raw_data = data_utils.load_dataset()
    except FileNotFoundError:
        raw_data = data_utils.create_dataset(n_samples=1000)

    return raw_data

In [4]:
raw_data = get_raw_data("data")

Data saved in data/data.pickle


In [5]:
coreset = Coreset()

coreset_vectors, coreset_weights = coreset.get_best_coresets(
    data_vectors=raw_data,
    number_of_runs=number_of_runs,
    coreset_numbers=number_of_qubits,
    size_vec_list=size_vec_list,
)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:8

In [6]:
coreset_vectors = normalize_np(coreset_vectors, centralize = True)
coreset_weights = normalize_np(coreset_weights, centralize = True)

In [7]:
coreset_vectors

array([[-0.19952738, -0.23312016],
       [-0.25687376,  0.13669904],
       [ 1.        ,  0.40811781],
       [-0.10877243, -1.        ],
       [-0.43482643,  0.6883033 ]])

In [8]:
# helper functions
def get_mean(coreset):
    dim = len(coreset[0])
    size_coreset = len(coreset)
    mu = np.zeros(dim)
    for i in range(size_coreset):
        mu += coreset[i]
    return mu/size_coreset
    
def get_weighted_mean(coreset, weights):
    dim = len(coreset[0])
    size_coreset = len(coreset)
    mu = np.zeros(dim)
    for i in range(size_coreset):
        mu += coreset[i]*weights[i]
    return mu/sum(weights)

def get_scatter_matrix(coreset):
    dim = len(coreset[0])
    size_coreset = len(coreset)
    T = np.zeros((dim,dim))
    mu = get_mean(coreset)
    for i in range(size_coreset):
        T += np.outer((coreset[i] - mu),(coreset[i] - mu))
    return T
    
def get_weighted_scatter_matrix(coreset, weights):
    dim = len(coreset[0])
    size_coreset = len(coreset)
    T = np.zeros((dim,dim))
    mu = get_weighted_mean(coreset, weights)
    for i in range(size_coreset):
        T += weights[i]*np.outer((coreset[i] - mu),(coreset[i] - mu))
    return T

def get_matrix_inverse(matrix):
    return inv(matrix)

In [9]:
def Z_i(i, length):
    """ 
    if index i is in the range 0, ..., length-1, the function returns the operator Z_i
    else: the funtion returns the pauli string consisting of pauli I's only
    length is the number of pauli operators tensorised
    """
    pauli_string = ""
    for j in range(length):
        if i == j:
            pauli_string += "Z"
        else:
            pauli_string += "I"
    return pauli_string

def Z_ij(i, j, length):
    pauli_string = ""
    if i == j:
        pauli_string = Z_i(-1, length) # return 'II...II'
    else:
        for k in range(length):
            if k == i or k == j:
                pauli_string += "Z"
            else:
                pauli_string += "I"
    return pauli_string

In [10]:
def create_hamiltonian(coreset, weights):

    paulis = []
    pauli_weights = []
    
    T_inv = inv(get_weighted_scatter_matrix(coreset, weights))

    W = sum(weights)

    for i in range(len(coreset)):
        paulis += [Z_i(-1, len(coreset))]
        pauli_weights += [weights[i]**2*np.dot(coreset[i], np.dot(T_inv, coreset[i]))]
    
        for l in range(len(coreset)):
            paulis += [Z_ij(i,l,len(coreset))]
            pauli_weights += [-2*weights[l]*weights[i]**2*np.dot(coreset[i], np.dot(T_inv, coreset[i]))/W]
            
    for j in range(len(coreset)):
        for i in range(j):
            paulis += [Z_ij(i,j,len(coreset))]
            pauli_weights += [2*weights[i]*weights[j]*np.dot(coreset[i], np.dot(T_inv, coreset[j]))]
            for l in range(len(coreset)):
                paulis += [Z_ij(i,l,len(coreset))]
                pauli_weights += [-2*weights[l]*weights[i]*weights[j]*np.dot(coreset[i], np.dot(T_inv, coreset[j]))/W]
                paulis += [Z_ij(j,l,len(coreset))]
                pauli_weights += [-2*weights[l]*weights[i]*weights[j]*np.dot(coreset[i], np.dot(T_inv, coreset[j]))/W]
            
            
    pauli_op = [([pauli,weight]) for pauli,weight in zip(paulis,pauli_weights)]
    
    
    return pauli_op

In [11]:
pauli_op = create_hamiltonian(coreset_vectors, coreset_weights)

In [41]:

hamiltonian = 0
for op in pauli_op:
    operator_string = op[0]
    coeff = op[1]
    operator = 1
    for i in range(len(operator_string)):
        op_i = operator_string[i]
        if op_i == "Z":
            operator *= spin.z(i) 
    print(hamiltonian)
    hamiltonian += coeff * operator
return -hamiltonian

0
[1.39995252e-16]
[0.27088006]


UFuncTypeError: Cannot cast ufunc 'add' output from dtype('O') to dtype('float64') with casting rule 'same_kind'

In [46]:
H = 0

(-0.504386,-0) Z0Z1


In [35]:
hamiltonian

array([0.27088006])

In [40]:
operator_string

'ZZIII'

In [42]:
i

4

In [43]:
hamiltonian = 5.907 - 2.1433 * spin.x(0) * spin.x(1) - 2.1433 * spin.y(
    0) * spin.y(1) + .21829 * spin.z(0) - 6.125 * spin.z(1)

In [45]:
print(hamiltonian)

(5.907,0) I0I1 + (-2.1433,-0) X0X1 + (-2.1433,-0) Y0Y1 + (0.21829,0) Z0I1 + (-6.125,-0) I0Z1
